<a href="https://colab.research.google.com/github/tomyhrdnsyh/Analisis-Sentimen-Twitter-Vaksin-Covid-19/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import string
import re

# import word tokenize & FreqDist from NLTK
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# import Sastrawi package
! pip install Sastrawi
! pip install swifter

In [10]:
TWEET_DATA = pd.read_csv("/content/1000_DATASET.csv", usecols=['komentar', 'sentimen'], encoding = "ISO-8859-1")
TWEET_DATA['komentar'].str.encode('ascii', 'ignore')
TWEET_DATA

,komentar,sentimen
0,Pak saya mau nikah pak thn ini.tapi wna belum ...,Negatif
1,"Tetap semangat pak Jokowi,mempertahankan negar...",Positif
2,Pak jokowi ... tolong lah psbb nya di selesaik...,Negatif
3,Sehat terus ya pak,Positif
4,Pak gimana mw berjuang.. suami saya udh gak ke...,Negatif
...,...,...
995,Kalau Bisa Pak @jokowi Jgn Terlalu Sibuk Denga...,Negatif
996,Mudah2an tenaga vaksin nya cukup untuk 270 jt ...,Positif
997,Terima kasih Bpk Presiden... semangatya Pak,Positif
998,Kaltim samarinda hadir salam sehat smg lancar,Positif


**Case Folding**

In [11]:
# -------- Case Folding ----------

TWEET_DATA['Tweet'] = TWEET_DATA['komentar'].str.lower()
TWEET_DATA

,komentar,sentimen,Tweet
0,Pak saya mau nikah pak thn ini.tapi wna belum ...,Negatif,pak saya mau nikah pak thn ini.tapi wna belum ...
1,"Tetap semangat pak Jokowi,mempertahankan negar...",Positif,"tetap semangat pak jokowi,mempertahankan negar..."
2,Pak jokowi ... tolong lah psbb nya di selesaik...,Negatif,pak jokowi ... tolong lah psbb nya di selesaik...
3,Sehat terus ya pak,Positif,sehat terus ya pak
4,Pak gimana mw berjuang.. suami saya udh gak ke...,Negatif,pak gimana mw berjuang.. suami saya udh gak ke...
...,...,...,...
995,Kalau Bisa Pak @jokowi Jgn Terlalu Sibuk Denga...,Negatif,kalau bisa pak @jokowi jgn terlalu sibuk denga...
996,Mudah2an tenaga vaksin nya cukup untuk 270 jt ...,Positif,mudah2an tenaga vaksin nya cukup untuk 270 jt ...
997,Terima kasih Bpk Presiden... semangatya Pak,Positif,terima kasih bpk presiden... semangatya pak
998,Kaltim samarinda hadir salam sehat smg lancar,Positif,kaltim samarinda hadir salam sehat smg lancar


 **Tokenizing**

In [13]:
# -------- Tokenizing -----------
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', " ")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(remove_tweet_special)
TWEET_DATA

,komentar,sentimen,Tweet
0,Pak saya mau nikah pak thn ini.tapi wna belum ...,Negatif,pak saya mau nikah pak thn ini.tapi wna belum ...
1,"Tetap semangat pak Jokowi,mempertahankan negar...",Positif,"tetap semangat pak jokowi,mempertahankan negar..."
2,Pak jokowi ... tolong lah psbb nya di selesaik...,Negatif,pak jokowi ... tolong lah psbb nya di selesaik...
3,Sehat terus ya pak,Positif,sehat terus ya pak
4,Pak gimana mw berjuang.. suami saya udh gak ke...,Negatif,pak gimana mw berjuang.. suami saya udh gak ke...
...,...,...,...
995,Kalau Bisa Pak @jokowi Jgn Terlalu Sibuk Denga...,Negatif,kalau bisa pak jgn terlalu sibuk dengan vaksin...
996,Mudah2an tenaga vaksin nya cukup untuk 270 jt ...,Positif,mudah2an tenaga vaksin nya cukup untuk 270 jt ...
997,Terima kasih Bpk Presiden... semangatya Pak,Positif,terima kasih bpk presiden... semangatya pak
998,Kaltim samarinda hadir salam sehat smg lancar,Positif,kaltim samarinda hadir salam sehat smg lancar


**Remove Unique Character**

In [14]:
def remove_unique_character(text):
    text = re.sub(r"\d+", "", text) #remove number
    text = text.translate(str.maketrans("","",string.punctuation)) #remove punctuation/tanda baca
    text = text.strip() #remove whitespace leading & trailing
    text = re.sub('\s+',' ',text) #remove multiple whitespace into single whitespace
    text = re.sub(r"\b[a-zA-Z]\b", "", text) #remove single character
    return  text

TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(remove_unique_character)
TWEET_DATA

,komentar,sentimen,Tweet
0,Pak saya mau nikah pak thn ini.tapi wna belum ...,Negatif,pak saya mau nikah pak thn initapi wna belum b...
1,"Tetap semangat pak Jokowi,mempertahankan negar...",Positif,tetap semangat pak jokowimempertahankan negara...
2,Pak jokowi ... tolong lah psbb nya di selesaik...,Negatif,pak jokowi tolong lah psbb nya di selesaikan s...
3,Sehat terus ya pak,Positif,sehat terus ya pak
4,Pak gimana mw berjuang.. suami saya udh gak ke...,Negatif,pak gimana mw berjuang suami saya udh gak kerj...
...,...,...,...
995,Kalau Bisa Pak @jokowi Jgn Terlalu Sibuk Denga...,Negatif,kalau bisa pak jgn terlalu sibuk dengan vaksin...
996,Mudah2an tenaga vaksin nya cukup untuk 270 jt ...,Positif,mudahan tenaga vaksin nya cukup untuk jt wni
997,Terima kasih Bpk Presiden... semangatya Pak,Positif,terima kasih bpk presiden semangatya pak
998,Kaltim samarinda hadir salam sehat smg lancar,Positif,kaltim samarinda hadir salam sehat smg lancar


In [15]:
# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

TWEET_DATA['tweet_tokens'] = TWEET_DATA['Tweet'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
TWEET_DATA

Tokenizing Result : 



,komentar,sentimen,Tweet,tweet_tokens
0,Pak saya mau nikah pak thn ini.tapi wna belum ...,Negatif,pak saya mau nikah pak thn initapi wna belum b...,"[pak, saya, mau, nikah, pak, thn, initapi, wna..."
1,"Tetap semangat pak Jokowi,mempertahankan negar...",Positif,tetap semangat pak jokowimempertahankan negara...,"[tetap, semangat, pak, jokowimempertahankan, n..."
2,Pak jokowi ... tolong lah psbb nya di selesaik...,Negatif,pak jokowi tolong lah psbb nya di selesaikan s...,"[pak, jokowi, tolong, lah, psbb, nya, di, sele..."
3,Sehat terus ya pak,Positif,sehat terus ya pak,"[sehat, terus, ya, pak]"
4,Pak gimana mw berjuang.. suami saya udh gak ke...,Negatif,pak gimana mw berjuang suami saya udh gak kerj...,"[pak, gimana, mw, berjuang, suami, saya, udh, ..."
...,...,...,...,...
995,Kalau Bisa Pak @jokowi Jgn Terlalu Sibuk Denga...,Negatif,kalau bisa pak jgn terlalu sibuk dengan vaksin...,"[kalau, bisa, pak, jgn, terlalu, sibuk, dengan..."
996,Mudah2an tenaga vaksin nya cukup untuk 270 jt ...,Positif,mudahan tenaga vaksin nya cukup untuk jt wni,"[mudahan, tenaga, vaksin, nya, cukup, untuk, j..."
997,Terima kasih Bpk Presiden... semangatya Pak,Positif,terima kasih bpk presiden semangatya pak,"[terima, kasih, bpk, presiden, semangatya, pak]"
998,Kaltim samarinda hadir salam sehat smg lancar,Positif,kaltim samarinda hadir salam sehat smg lancar,"[kaltim, samarinda, hadir, salam, sehat, smg, ..."


In [16]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

TWEET_DATA['tweet_tokens_fdist'] = TWEET_DATA['tweet_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
TWEET_DATA['tweet_tokens_fdist'].head().apply(lambda x : x.most_common())
TWEET_DATA

Frequency Tokens : 



,komentar,sentimen,Tweet,tweet_tokens,tweet_tokens_fdist
0,Pak saya mau nikah pak thn ini.tapi wna belum ...,Negatif,pak saya mau nikah pak thn initapi wna belum b...,"[pak, saya, mau, nikah, pak, thn, initapi, wna...","{'pak': 2, 'saya': 1, 'mau': 1, 'nikah': 1, 't..."
1,"Tetap semangat pak Jokowi,mempertahankan negar...",Positif,tetap semangat pak jokowimempertahankan negara...,"[tetap, semangat, pak, jokowimempertahankan, n...","{'tetap': 1, 'semangat': 1, 'pak': 1, 'jokowim..."
2,Pak jokowi ... tolong lah psbb nya di selesaik...,Negatif,pak jokowi tolong lah psbb nya di selesaikan s...,"[pak, jokowi, tolong, lah, psbb, nya, di, sele...","{'pak': 1, 'jokowi': 1, 'tolong': 1, 'lah': 1,..."
3,Sehat terus ya pak,Positif,sehat terus ya pak,"[sehat, terus, ya, pak]","{'sehat': 1, 'terus': 1, 'ya': 1, 'pak': 1}"
4,Pak gimana mw berjuang.. suami saya udh gak ke...,Negatif,pak gimana mw berjuang suami saya udh gak kerj...,"[pak, gimana, mw, berjuang, suami, saya, udh, ...","{'pak': 1, 'gimana': 1, 'mw': 1, 'berjuang': 1..."
...,...,...,...,...,...
995,Kalau Bisa Pak @jokowi Jgn Terlalu Sibuk Denga...,Negatif,kalau bisa pak jgn terlalu sibuk dengan vaksin...,"[kalau, bisa, pak, jgn, terlalu, sibuk, dengan...","{'kalau': 1, 'bisa': 2, 'pak': 2, 'jgn': 1, 't..."
996,Mudah2an tenaga vaksin nya cukup untuk 270 jt ...,Positif,mudahan tenaga vaksin nya cukup untuk jt wni,"[mudahan, tenaga, vaksin, nya, cukup, untuk, j...","{'mudahan': 1, 'tenaga': 1, 'vaksin': 1, 'nya'..."
997,Terima kasih Bpk Presiden... semangatya Pak,Positif,terima kasih bpk presiden semangatya pak,"[terima, kasih, bpk, presiden, semangatya, pak]","{'terima': 1, 'kasih': 1, 'bpk': 1, 'presiden'..."
998,Kaltim samarinda hadir salam sehat smg lancar,Positif,kaltim samarinda hadir salam sehat smg lancar,"[kaltim, samarinda, hadir, salam, sehat, smg, ...","{'kaltim': 1, 'samarinda': 1, 'hadir': 1, 'sal..."


**STOPWORD**

Mengimport stopword dari library nltk

In [17]:
nltk.download('stopwords')
from nltk.corpus import stopwords
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
758


Menambah list stopword secara manual

In [18]:
# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'man'])
len(list_stopwords)

798

**Remove Stopword**

In [19]:
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

TWEET_DATA['tweet_tokens_WSW'] = TWEET_DATA['tweet_tokens'].apply(stopwords_removal)
# TWEET_DATA['tweet_tokens_WSW'].head()

In [23]:
normalizad_word = pd.read_excel("/content/key_norm_revisi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

TWEET_DATA['tweet_normalized'] = TWEET_DATA['tweet_tokens_WSW'].apply(normalized_term)
TWEET_DATA['tweet_normalized']

0      [nikah, tahun, initapi, wna, boleh, masuk, ind...
1               [semangat, jokowimempertahankan, negara]
2      [jokowi, tolong, psbb, selesaikan, secepatnya,...
3                                                [sehat]
4      [bagaimana, mau, berjuang, suami, sudah, kerja...
                             ...                        
995    [sibuk, vaksinbiar, menkes, tugasnyalebih, vis...
996                   [mudahan, tenaga, vaksin, jt, wni]
997         [terima, kasih, bapak, presiden, semangatya]
998    [kaltim, samarinda, hadir, salam, sehat, semog...
999                                             [vaksin]
Name: tweet_normalized, Length: 1000, dtype: object

# **STEMMING**

In [24]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

In [27]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in TWEET_DATA['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

3507
------------------------


In [30]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    # print(term, ":", term_dict[term])



In [34]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]


TWEET_DATA['tweet_tokens_stemmed'] = TWEET_DATA['tweet_normalized'].swifter.apply(get_stemmed_term)
TWEET_DATA['tweet_tokens_stemmed']

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

0      [nikah, tahun, initapi, wna, boleh, masuk, ind...
1               [semangat, jokowimempertahankan, negara]
2      [jokowi, tolong, psbb, selesai, cepat, butuh, ...
3                                                [sehat]
4      [bagaimana, mau, juang, suami, sudah, kerja, p...
                             ...                        
995    [sibuk, vaksinbiar, menkes, tugasnyalebih, vis...
996                     [mudah, tenaga, vaksin, jt, wni]
997         [terima, kasih, bapak, presiden, semangatya]
998    [kaltim, samarinda, hadir, salam, sehat, moga,...
999                                             [vaksin]
Name: tweet_tokens_stemmed, Length: 1000, dtype: object

In [ ]:
# Save ke CSV
path = "/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/Output/"
TWEET_DATA.to_csv(path+"Tweet_Preprocessing.csv")